In [1]:
from __future__ import division
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
from sklearn.decomposition import PCA
import pandas as pd
import math
from numpy import random
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold

In [2]:
X = np.genfromtxt('Data_OpenCV/Features_X.csv',delimiter=',')
Y = np.genfromtxt('Data_OpenCV/Y_labels.csv',delimiter=',')

In [3]:
print(X.shape)
print(Y.shape)

(13493, 150)
(13493,)


In [4]:
print(Y.data[0:5])

In [5]:
def prueba_svm(ker='linear',conf=0.001,gam=0.0):
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    porc_vectores = np.zeros(Folds)
    j = 0
    
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        scaler = preprocessing.StandardScaler().fit(Xtrain)
        Xtrain = scaler.transform(Xtrain)
        Xtest = scaler.transform(Xtest)
        
        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        if ker == 'linear':
            modelo = SVC(kernel =ker, C=conf)
        else:
            modelo = SVC(kernel=ker, C= conf, gamma= gam)
        modelo.fit(Xtrain,Ytrain)


        #Validación
        Ytrain_pred = modelo.predict(Xtrain)
        Yest = modelo.predict(Xtest)

        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
        EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        porc_vectores[j] = len(modelo.support_vectors_)/len(Xtrain)
        j += 1

    print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
    print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
    
    eficiencia_val = np.mean(EficienciaVal)
    ic_val = np.std(EficienciaVal)
    eficiencia_train = np.mean(EficienciaTrain)
    ic_train = np.std(EficienciaTrain)
    porc_vect = np.mean(porc_vectores)
    
    return eficiencia_val,ic_val,eficiencia_train,ic_train,porc_vect

In [6]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types["Eficiencia en entrenamiento"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)

In [ ]:
#df_types.loc[3,"Eficiencia en validacion"] = "0.97077"
#df_types.loc[3,"Intervalo de confianza"] = "0.01548"
#df_types.loc[3, "% de Vectores de Soporte"] = "0.2620"
#df_types.sort_index(inplace=True)
j = 0
for i in df_types.index:
    print(i)
    kernel = i[0]
    c = i[1]
    gamma = i[2]    
    eficiencia_val,ic_val,eficiencia_train,ic_train,porc_vect=prueba_svm(ker=kernel,conf=c,gam=gamma)
    df_types.loc[i,"Eficiencia en validacion"] = str(eficiencia_val)
    df_types.loc[i,"Intervalo de confianza"] = str(ic_val)
    df_types.loc[i,"Eficiencia en entrenamiento"] = str(eficiencia_train)
    df_types.loc[i,"Intervalo de confianza"] = str(ic_train)
    df_types.loc[i, "% de Vectores de Soporte"] = str(porc_vect)
    j = j+1
    print(eficiencia_val,ic_val,eficiencia_train,ic_train,porc_vect)

('linear', 0.001, 0.0)
Eficiencia durante el entrenamiento = 0.4071741581504211+-0.0032109575967907925
Eficiencia durante la validación = 0.3862749420016797+-0.00744402795614879
0.3862749420016797 0.00744402795614879 0.4071741581504211 0.0032109575967907925 0.9863386056715598
('linear', 0.01, 0.0)
Eficiencia durante el entrenamiento = 0.41621583590060884+-0.0033881910177456474
Eficiencia durante la validación = 0.3841256079916334+-0.008541305587442675
0.3841256079916334 0.008541305587442675 0.41621583590060884 0.0033881910177456474 0.982410627758918
('linear', 0.1, 0.0)
Eficiencia durante el entrenamiento = 0.4177474930735317+-0.0030653956002504377
Eficiencia durante la validación = 0.38531116641427593+-0.008245213466106361
0.38531116641427593 0.008245213466106361 0.4177474930735317 0.0030653956002504377 0.9921440661464541
('linear', 1.0, 0.0)
Eficiencia durante el entrenamiento = 0.41789577056740207+-0.0035540645750752006
Eficiencia durante la validación = 0.3850145845938958+-0.008123